In [1]:
import os
from numpy import *

glib_path = os.path.abspath("../datapacks/Glib")
data_path = f"{glib_path}/data"
module_path = data_path + "/{m}/functions"

# `glib.block` & `Glib.item` search trees


In [ ]:
os.system(f"python ./block_converter/main.py")

# `_` functions

Generate "_" functions that allow to determine which modules are active or not. They also allow to fix the autocompletion for the user.

In [2]:
exclude_module = ["glib", "glib.core", "minecraft"]

# Generating all "_" functions
for module in os.listdir(data_path):
    if module in exclude_module: continue
    module_type, module_name = module.split(".")
    if not os.path.isdir(module_path.format(m=module)): os.makedirs(module_path.format(m=module))

    with open(f"{module_path.format(m=module)}/_.mcfunction","w") as f:
        msg = {"text":f"[{module} documentation]","color":"dark_aqua","clickEvent":{"action":"open_url","value":f"https://glibs.rtfd.io/en/latest/{module_name}.html"},"hoverEvent":{"action":"show_text","contents":"Click to open URL"}}
        f.write(f"tellraw @s [{msg}]\n".replace("'",'"'))
        f.write(f"scoreboard players set {module} glib.activeModule 1")

# Generating glib.core:load.mcfunction
with open(f"{data_path}/glib.core/functions/load.mcfunction", "r") as old, open(f"{data_path}/glib.core/functions/load_tmp.mcfunction", "w") as new:
    for line in old:
        new.write(line)
        if line.startswith("# Module list"): break
    for module in os.listdir(f"{glib_path}/data"):
        if module in exclude_module: continue
        new.write(f"function {module}:_\n")
os.remove(f"{data_path}/glib.core/functions/load.mcfunction")
os.rename(f"{data_path}/glib.core/functions/load_tmp.mcfunction", f"{data_path}/glib.core/functions/load.mcfunction")

# Menu > Debug

Allow to add/remove tags to debug systems

In [2]:
import libs.menu as menu

exclude_module = ["glib", "glib.core", "minecraft"]

modules = []
modules.append(("Debug stick","tag","glib.debug.stick"))
modules.append((" ","text",None))

for module in os.listdir(data_path):
    if module in exclude_module: continue
    modules.append((module, "tag", f"glib.debug.{module.split('.')[1]}"))

menu.paged_menu(
    dest = "../datapacks/Glib/data/glib.core/functions/menu/debug.mcfunction",
    title="Glib Menu / Debug",
    menu_tag = "glib.menu.debug",
    parent_menu_tag = "glib.menu",
    items = modules,
    maxLine=20
)
